In [1479]:
import plotly as pl
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas as pd
import seaborn as sns
from dash_bootstrap_templates import load_figure_template

app = Dash(__name__, external_stylesheets=[dbc.themes.YETI])

load_figure_template("YETI")

In [1480]:
resorts = (
    pd.read_csv("Data/Ski Resorts/resorts.csv", encoding="ISO-8859-1")
    .assign(
        country_elevation_rank=lambda x: x.groupby("Country", as_index=False)[
            "Highest point"].rank(ascending=False),
        country_price_rank=lambda x: x.groupby("Country", as_index=False)[
            "Price"].rank(ascending=False),
        country_slope_rank=lambda x: x.groupby("Country", as_index=False)[
            "Total slopes"].rank(ascending=False),
        country_cannon_rank=lambda x: x.groupby("Country", as_index=False)[
            "Snow cannons"].rank(ascending=False),
    ))
resorts.head()

,ID,Resort,Latitude,Longitude,Country,Continent,Price,Season,Highest point,Lowest point,...,Total lifts,Lift capacity,Child friendly,Snowparks,Nightskiing,Summer skiing,country_elevation_rank,country_price_rank,country_slope_rank,country_cannon_rank
0,1,Hemsedal,60.928244,8.383487,Norway,Europe,46,November - May,1450,620,...,21,22921,Yes,Yes,Yes,No,2.0,1.5,4.0,2.0
1,2,Geilosiden Geilo,60.534526,8.206372,Norway,Europe,44,November - April,1178,800,...,24,14225,Yes,Yes,Yes,No,4.0,6.0,6.5,4.0
2,3,Golm,47.057810,9.828167,Austria,Europe,48,December - April,2110,650,...,11,16240,Yes,No,No,No,37.0,31.5,63.0,45.0
3,4,Red Mountain Resort-Rossland,49.105520,-117.846280,Canada,North America,60,December - April,2075,1185,...,8,9200,Yes,Yes,Yes,No,13.0,13.0,5.0,15.0
4,5,Hafjell,61.230369,10.529014,Norway,Europe,45,November - April,1030,195,...,18,21060,Yes,Yes,Yes,No,9.0,3.5,3.0,3.0


In [1481]:
resorts['Snowparks'] = resorts['Snowparks'].map({'Yes': True, 'No': False})
resorts['Nightskiing'] = resorts['Nightskiing'].map({'Yes': True, 'No': False})
resorts['Summer skiing'] = resorts['Summer skiing'].map({'Yes': True, 'No': False})


In [1482]:

def continents_to_countries(option, df):
    continents_hashmap = {}
    continents = list(df['Continent'].unique())

    for continent in continents:
        con = df[df['Continent'] == continent]
        countries = list(con['Country'].unique())
        continents_hashmap[continent] = countries


    return continents_hashmap.get(option)

In [1483]:
app.layout = dbc.Container([
    dcc.Markdown(id="Markdown Title"),
    html.Br(),

    dcc.Tabs([
        dcc.Tab(
            [
            dbc.Row([html.Br()]),
                dbc.Row([html.H1('Global Ski Resort Density',
                        style={"text-align": 'center'})]),
            dbc.Row([html.Br()]),
            dbc.Row([
                dbc.Col([
                    dbc.Row([dcc.RangeSlider(
                    id='price',
                    min=resorts['Price'].min(),
                    max=resorts['Price'].max(),
                    step=15,
                    value=[0, resorts['Price'].max()],
                    className="dbc"
                )]),
                dbc.Row([html.Br()]),
                dbc.Row([dcc.Checklist(
                    id='summer_skiing',
                    options=[{'label': 'Summer Skiing', 'value': True}],
                    value=[]
                )]),
                dbc.Row([dcc.Checklist(
                    id='night_skiing',
                    options=[{'label': 'Night Skiing', 'value': True}],
                    value=[]
                )]),
                dbc.Row([dcc.Checklist(
                    id='snowparks',
                    options=[{'label': 'Snowparks', 'value': True}],
                    value=[]
                )])],
                width=3),
                dbc.Col(dbc.Card(dcc.Graph(id='ski_slope_density')))
            ])],
            label = "World Map"
        ),
        dcc.Tab(
            [
            dbc.Row([html.Br()]),
            dbc.Row([html.H1(id='tab2_title', style={"text-align": 'center'})]),
            dbc.Row([html.Br()]),
            dbc.Row([
                dbc.Col([
                    dcc.Markdown('Select Continent:'),
                    dcc.Dropdown(
                        id='continent_dropdown',
                        options=resorts['Continent'].unique(),
                        value='Europe',
                        className="dbc"
                        ),
                    html.Br(),
                    dcc.Markdown('Select Country:'),
                    dcc.Dropdown(
                        id='country_dropdown', 
                        value='United Kingdom', 
                        className="dbc"
                        ),
                    html.Br(),
                    dcc.Markdown('Select Metric:'),
                    dcc.Dropdown(
                        id='user_metric',
                        options=['Price', 'Highest point', 'Lowest point', 'Lowest point', 'Beginner slopes',
                                'Intermediate slopes', 'Difficult slopes', 'Total slopes',
                                'Longest run'],
                        value='Price',
                        className="dbc"
                    

                    )
                ], width=3),
                dbc.Col(dbc.Card(dcc.Graph(id='top10_by_metric', hoverData={'points': [{'customdata': ['Nevis Range']}]})), width=6),
                dbc.Col([
                    dbc.Row(dbc.Card(html.H2(id='resort_title', style={'fontSize': 30}))),
                    dbc.Row(html.Br()),
                    dbc.Row([dbc.Col(dbc.Card(html.H3(id='kpi_price', style={'fontSize': 20}))),
                            dbc.Col(dbc.Card(html.H3(id='kpi_elevation', style={'fontSize': 20})))]),
                    dbc.Row(html.Br()),
                    dbc.Row([dbc.Col(dbc.Card(html.H3(id='kpi_slope', style={'fontSize': 20}))),
                            dbc.Col(dbc.Card(html.H3(id='kpi_cannon', style={'fontSize': 20})))]),
                    ], 
                    width=3,
                    style={"text-align": 'center'}
                )
            ]),
            ],
            label="Bar Chart"
            
        ),
        
    ],
    className='dbc')
    
])

In [1484]:
@app.callback(Output("country_dropdown", "options"), 
              Input('continent_dropdown', 'value'))
def set_country_option(continent):
    countries = continents_to_countries(continent, resorts)
    return countries

In [1485]:
@app.callback(Output("ski_slope_density", "figure"), 
              Input('price', 'value'), 
              Input('summer_skiing', 'value'), 
              Input('night_skiing', 'value'),
              Input('snowparks', 'value'))
def state_graph(price, summer_skiing, night_skiing, snowparks):

    middle_europe = {'lat': 54.15, 'lon': 18.64}

    df = resorts.query(
        "@price[0] <= Price <= @price[1]")
    
    if True in summer_skiing:
        df = df.loc[df['Summer skiing'] == True]

    if True in night_skiing:
        df = df.loc[df['Nightskiing'] == True]

    if True in snowparks:
        df = df.loc[df['Snowparks'] == True]


    fig = px.density_map(
        df,
        lat="Latitude",
        lon="Longitude",
        z='Total slopes',
        radius=15,
        center=middle_europe,
        zoom=2
    )
    return fig

In [1486]:
@app.callback(Output("top10_by_metric", "figure"),
              Output("tab2_title", 'children'),
              Input('user_metric', 'value'),
              Input('country_dropdown', 'value'))
def resort_details(metric, country):

    topn = 10
    
    if not country:
        raise PreventUpdate
    
    title = f"Top 10 Resorts in {country} by {metric}"
    country_resorts = resorts[resorts['Country'] == country]
    top10 = country_resorts.sort_values(f'{metric}', ascending=False).head(topn)
    fig = px.bar(
        top10,
        x='Resort',
        y=f'{metric}',
        hover_name='Resort',
        custom_data=['Resort']
    )
    fig.update_xaxes(showticklabels=False)
    return fig, title

In [1487]:

@app.callback(Output("resort_title", 'children'),
              Output("kpi_price", 'children'),
              Output("kpi_elevation", 'children'),
              Output("kpi_slope", 'children'),
              Output("kpi_cannon", 'children'),
              Input('top10_by_metric', 'hoverData'))
def report_card(hoverData):

    resort_name = hoverData['points'][0]['customdata'][0]

    resort = resorts[resorts['Resort'] == resort_name]

    title = f'{resort_name}'
    price = f"Price Ranking: {str(int(resort['country_price_rank'].iloc[0]))}"
    elevation = f"Elevation Ranking: {str(int(resort['country_elevation_rank'].iloc[0]))}"
    slope = f"Slope Ranking: {str(int(resort['country_slope_rank'].iloc[0]))}"
    cannon = f"Cannons Ranking: {str(int(resort['country_cannon_rank'].iloc[0]))}"

    return title, price, elevation, slope, cannon

In [1488]:
if __name__ == '__main__':
    app.run(port=8052)